# Path explorer

This notebook contains code related to the definition and exploration of the different states that the `MED` can take at different samples in time.

- Directed graph build by defining nodes / vertices that are the states that the FSM of the system can take, and repeated at each time step. Also, the edges (representing transitions between states) connecting these nodes are obtained from the FSM itself, so only valid transitions from the states at each step are considered.
- Path exploration. In order to obtain every possible path for any given initial state and a prediction horizon (max number of steps to evaluate). A Deep First Search (DFS) is performed in a recursive function to find every possibility. The result is a list with every possible path or trajectory for the given initial states. Note that increasing the number of steps can lead to the "tree explosion" due to the exponential growth that takes place in these kind of systems.
- The visualization of these two aspects, is then visualized.

Expected result:

![expected result](../docs/attachments/path_viz_med.png)

## Tasks
- Import existing computations for the individual systems
- Combine them to have the complete system available
- Visualize paths interactively

In [36]:
from typing import Literal
from enum import Enum
from pathlib import Path
from ipywidgets import interact, fixed
import pandas as pd
from phd_visualizations import save_figure
from solarmed_modeling.fsms import SolarMedState
from solarmed_modeling.fsms.sfts import get_sf_ts_individual_states
from solarmed_modeling.fsms.utils import convert_to
from solarmed_modeling.fsms.utils import SupportedSubsystemsStatesMapping
from solarmed_modeling.visualization.fsm import (
    Node,
    generate_nodes_df
)
from solarmed_modeling.visualization.fsm.state_evolution import(
    plot_state_graph,
    get_coordinates_edge,
)
from solarmed_optimization.path_explorer.utils import import_results
from solarmed_optimization.visualization import highlight_path

%load_ext autoreload
%autoreload 2

data_path: Path = Path("../results")
n_horizon:int = 5
sample_time = 1
parameters_sfts = dict(sample_time= sample_time)
parameters_med = dict(
    sample_time=sample_time,
    vacuum_duration_time=3,
    brine_emptying_time=1,
    startup_duration_time=1
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
paths_sfts_df = import_results(data_path, system="SFTS", n_horizon=n_horizon, params=parameters_sfts)
paths_med_df = import_results(paths_path=data_path, system="MED", n_horizon=n_horizon, params=parameters_med)
# paths_sfts_str_df = import_results(data_path, system="SFTS", n_horizon=n_horizon, params=parameters_sfts, return_format="name")
# paths_med_str_df = import_results(paths_path=data_path, system="MED", n_horizon=n_horizon, params=parameters_med, return_format="name")
# paths_sfts_values_df = import_results(data_path, system="SFTS", n_horizon=n_horizon, params=parameters_sfts, return_format="value")
# paths_med_values_df = import_results(paths_path=data_path, system="MED", n_horizon=n_horizon, params=parameters_med, return_format="value")


display(paths_sfts_df)
# display(paths_sfts_str_df)

nodes_med_df = generate_nodes_df(n_horizon, "MED")
nodes_sfts_df = generate_nodes_df(n_horizon, "SFTS")

display(nodes_sfts_df)


,0,1,2,3,4
0,SfTsState.RECIRCULATING_TS,SfTsState.IDLE,SfTsState.RECIRCULATING_TS,SfTsState.IDLE,SfTsState.RECIRCULATING_TS
1,SfTsState.SF_HEATING_TS,SfTsState.IDLE,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS
2,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.IDLE
3,SfTsState.HEATING_UP_SF,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.IDLE,SfTsState.IDLE
4,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS,SfTsState.SF_HEATING_TS,SfTsState.SF_HEATING_TS,SfTsState.SF_HEATING_TS
...,...,...,...,...,...
174,SfTsState.IDLE,SfTsState.IDLE,SfTsState.IDLE,SfTsState.RECIRCULATING_TS,SfTsState.IDLE
175,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS
176,SfTsState.RECIRCULATING_TS,SfTsState.IDLE,SfTsState.RECIRCULATING_TS,SfTsState.RECIRCULATING_TS,SfTsState.RECIRCULATING_TS
177,SfTsState.HEATING_UP_SF,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.IDLE


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SfTsState.IDLE,step000_0,0,IDLE,0,0.0
1,0,SfTsState.HEATING_UP_SF,step000_1,1,HEATING_UP_SF,0,1.0
2,0,SfTsState.SF_HEATING_TS,step000_2,2,SF_HEATING_TS,0,2.0
3,0,SfTsState.RECIRCULATING_TS,step000_3,3,RECIRCULATING_TS,0,3.0
4,1,SfTsState.IDLE,step001_0,0,IDLE,1,0.0
5,1,SfTsState.HEATING_UP_SF,step001_1,1,HEATING_UP_SF,1,1.0
6,1,SfTsState.SF_HEATING_TS,step001_2,2,SF_HEATING_TS,1,2.0
7,1,SfTsState.RECIRCULATING_TS,step001_3,3,RECIRCULATING_TS,1,3.0
8,2,SfTsState.IDLE,step002_0,0,IDLE,2,0.0
9,2,SfTsState.HEATING_UP_SF,step002_1,1,HEATING_UP_SF,2,1.0


### Heat generation and storage subsystem

In [16]:
system: Literal["SFTS", "MED"] = "SFTS"
states_enum: Enum = getattr(SupportedSubsystemsStatesMapping, system).value
paths_df = paths_sfts_df
nodes_df = nodes_sfts_df

fig = plot_state_graph(nodes_df, edges_df=None, system_title=system, Np=n_horizon, width=900)
path_values_df = paths_df.map(lambda state: convert_to(state, states_enum, return_format="value"))
max_group_size = path_values_df.groupby("0").size().max()

@interact(initial_state=[state for state in states_enum], option_idx=(0, max_group_size, 1), fig=fixed(fig))
def callback(initial_state=states_enum(0), option_idx=0, fig=fig):
    base_title = f"Directed graph of the operating modes evolution in the {system} system"
    highlight_path(
        fig, paths_df=paths_df, paths_values_df=path_values_df, selected_path_idx=option_idx, 
        initial_state=initial_state, nodes_df=nodes_df, base_title=base_title
    )
    
fig


interactive(children=(Dropdown(description='initial_state', options=(<SfTsState.IDLE: 0>, <SfTsState.HEATING_U…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': 'baa7a68b-c410-490c-9948-fe73d7cc1290',
              'x': [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
              'y': [0.0, 1.0, 2.0, 3.0, 0.0, 1.0, 2.0, 3.0, 0.0, 1.0, 2.0, 3.0,
             

### MED

In [8]:
system: Literal["SFTS", "MED"] = "MED"
states_enum: Enum = getattr(SupportedSubsystemsStatesMapping, system).value
paths_df = paths_med_df
nodes_df = nodes_med_df

fig = plot_state_graph(nodes_df, edges_df=None, system_title=system, Np=n_horizon, width=900)
path_values_df = paths_df.map(lambda state: convert_to(state, states_enum, return_format="value"))
max_group_size = path_values_df.groupby("0").size().max()

@interact(initial_state=[state for state in states_enum], option_idx=(0, max_group_size, 1), fig=fixed(fig))
def callback(initial_state=states_enum(0), option_idx=0, fig=fig):
    base_title = f"Directed graph of the operating modes evolution in the {system} system"
    highlight_path(
        fig, paths_df=paths_df, paths_values_df=path_values_df, selected_path_idx=option_idx, 
        initial_state=initial_state, nodes_df=nodes_df, base_title=base_title
    )
    
fig


interactive(children=(Dropdown(description='initial_state', options=(<MedState.OFF: 0>, <MedState.GENERATING_V…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#c061cb', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['OFF', 'GENERATING_VACUUM', 'IDLE', 'STARTING_UP', 'SHUTTING_DOWN',
                             'ACTIVE', 'OFF', 'GENERATING_VACUUM', 'IDLE', 'STARTING_UP',
                             'SHUTTING_DOWN', 'ACTIVE', 'OFF', 'GENERATING_VACUUM', 'IDLE',
                             'STARTING_UP', 'SHUTTING_DOWN', 'ACTIVE', 'OFF', 'GENERATING_VACUUM',
                             'IDLE', 'STARTING_UP', 'SHUTTING_DOWN', 'ACTIVE', 'OFF',
                             'GENERATING_VACUUM', 'IDLE', 'STARTING_UP', 'SHUTTING_DOWN', 'ACTIVE'],
                            dtype=object),
              'type': 'scatter',
              'uid': '69b2224a-ba06-46f5-ba81-b3cc929e3397',
              'x': [0, 0, 0, 0, 0, 0,

### SolarMED

In [9]:
paths: list[list[Enum]] = []
for _, sfts_path in paths_sfts_df.iterrows():
    for _, med_path in paths_med_df.iterrows():        
        # Forgive me lord for I am about to sin
        path_str = [f'{"".join(map(str, get_sf_ts_individual_states(sfts_state, "value")))}{med_state.value}' for sfts_state, med_state in zip(sfts_path.values, med_path.values)]
        path = [SolarMedState(state_str) for state_str in path_str]
        
        paths.append(path)

paths_solarmed_df = pd.DataFrame(paths, columns=[str(i) for i in range(len(paths[0]))])
display(paths_solarmed_df)

nodes_solarmed_df = generate_nodes_df(n_horizon, "SolarMED")
display(nodes_solarmed_df)


,0,1,2,3,4
0,SolarMedState.sf_IDLE_ts_ACTIVE_med_IDLE,SolarMedState.sf_IDLE_ts_IDLE_med_STARTING_UP,SolarMedState.sf_IDLE_ts_ACTIVE_med_ACTIVE,SolarMedState.sf_IDLE_ts_IDLE_med_ACTIVE,SolarMedState.sf_IDLE_ts_ACTIVE_med_SHUTTING_DOWN
1,SolarMedState.sf_IDLE_ts_ACTIVE_med_ACTIVE,SolarMedState.sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,SolarMedState.sf_IDLE_ts_ACTIVE_med_IDLE,SolarMedState.sf_IDLE_ts_IDLE_med_OFF,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF
2,SolarMedState.sf_IDLE_ts_ACTIVE_med_SHUTTING_DOWN,SolarMedState.sf_IDLE_ts_IDLE_med_IDLE,SolarMedState.sf_IDLE_ts_ACTIVE_med_IDLE,SolarMedState.sf_IDLE_ts_IDLE_med_STARTING_UP,SolarMedState.sf_IDLE_ts_ACTIVE_med_ACTIVE
3,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF,SolarMedState.sf_IDLE_ts_IDLE_med_OFF,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF,SolarMedState.sf_IDLE_ts_IDLE_med_OFF,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF
4,SolarMedState.sf_IDLE_ts_ACTIVE_med_ACTIVE,SolarMedState.sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF,SolarMedState.sf_IDLE_ts_IDLE_med_OFF,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF
...,...,...,...,...,...
12167,SolarMedState.sf_ACTIVE_ts_IDLE_med_OFF,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_OFF,SolarMedState.sf_IDLE_ts_IDLE_med_GENERATING_V...,SolarMedState.sf_IDLE_ts_ACTIVE_med_GENERATING...,SolarMedState.sf_IDLE_ts_ACTIVE_med_GENERATING...
12168,SolarMedState.sf_ACTIVE_ts_IDLE_med_ACTIVE,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_SHUTTING...,SolarMedState.sf_IDLE_ts_IDLE_med_IDLE,SolarMedState.sf_IDLE_ts_ACTIVE_med_IDLE,SolarMedState.sf_IDLE_ts_ACTIVE_med_OFF
12169,SolarMedState.sf_ACTIVE_ts_IDLE_med_IDLE,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_IDLE,SolarMedState.sf_IDLE_ts_IDLE_med_IDLE,SolarMedState.sf_IDLE_ts_ACTIVE_med_STARTING_UP,SolarMedState.sf_IDLE_ts_ACTIVE_med_ACTIVE
12170,SolarMedState.sf_ACTIVE_ts_IDLE_med_STARTING_UP,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_ACTIVE,SolarMedState.sf_IDLE_ts_IDLE_med_ACTIVE,SolarMedState.sf_IDLE_ts_ACTIVE_med_SHUTTING_DOWN,SolarMedState.sf_IDLE_ts_ACTIVE_med_IDLE


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SolarMedState.sf_IDLE_ts_IDLE_med_OFF,step000_000,000,sf_IDLE_ts_IDLE_med_OFF,0,0
1,0,SolarMedState.sf_IDLE_ts_IDLE_med_GENERATING_V...,step000_001,001,sf_IDLE_ts_IDLE_med_GENERATING_VACUUM,0,1
2,0,SolarMedState.sf_IDLE_ts_IDLE_med_IDLE,step000_002,002,sf_IDLE_ts_IDLE_med_IDLE,0,2
3,0,SolarMedState.sf_IDLE_ts_IDLE_med_STARTING_UP,step000_003,003,sf_IDLE_ts_IDLE_med_STARTING_UP,0,3
4,0,SolarMedState.sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,step000_004,004,sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,0,4
...,...,...,...,...,...,...,...
115,4,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_GENERATI...,step004_111,111,sf_ACTIVE_ts_ACTIVE_med_GENERATING_VACUUM,4,19
116,4,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_IDLE,step004_112,112,sf_ACTIVE_ts_ACTIVE_med_IDLE,4,20
117,4,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_STARTING_UP,step004_113,113,sf_ACTIVE_ts_ACTIVE_med_STARTING_UP,4,21
118,4,SolarMedState.sf_ACTIVE_ts_ACTIVE_med_SHUTTING...,step004_114,114,sf_ACTIVE_ts_ACTIVE_med_SHUTTING_DOWN,4,22


In [15]:
system: str = "SolarMED"

states_enum: Enum = SolarMedState
paths_df = paths_solarmed_df
nodes_df = nodes_solarmed_df
nodes_comp_dfs = [nodes_sfts_df, nodes_med_df]

fig = plot_state_graph(nodes_comp_dfs, edges_df=None, system_title=system, Np=n_horizon, width=900)
path_values_df = paths_df.map(lambda state: convert_to(state, states_enum, return_format="value"))
max_group_size = path_values_df.groupby("0").size().max()
shift = len(nodes_sfts_df[ nodes_sfts_df["step_idx"] == 0 ]["state"]) # Number of SFTS states
base_title = f"Directed graph of the operating modes evolution in the {system} system"

@interact(initial_state=[state for state in SolarMedState], option_idx=(0, max_group_size, 1), fig=fixed(fig))
def callback(initial_state=nodes_df.iloc[0]["state"], option_idx=0, fig=fig):
    highlight_path(
        fig, paths_df=paths_df, paths_values_df=path_values_df, selected_path_idx=option_idx, 
        initial_state=initial_state, nodes_df=nodes_df, base_title=base_title,
        nodes_comp_dfs=nodes_comp_dfs, shift=shift
    )
    
fig


interactive(children=(Dropdown(description='initial_state', options=(<SolarMedState.sf_IDLE_ts_IDLE_med_OFF: '…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS', 'IDLE',
                             'HEATING_UP_SF', 'SF_HEATING_TS', 'RECIRCULATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': '2c71ef9e-9886-486e-9d0e-0bbd007b5e8e',
              'x': [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
              'y': [0.0, 1.0, 2.0, 3.0, 0.0, 1.0, 2.0, 3.0, 0.0, 1.0, 2.0, 3.0,
             

# Old

In [4]:
# Build states/nodes/vertices dataframe

state_cls = SolarMED_State

base_list = [str(state.value) for state in state_cls]
N_nodes = Np*len(base_list)

result = [
    Node(
        step_idx=step_idx,
        state=state,
    ).model_dump()
    for step_idx in range(Np) for state in [state for state in state_cls]
]


nodes_sfts_df = pd.DataFrame([
    Node(step_idx=step_idx,state=state).model_dump()
    for step_idx in range(Np) for state in [state for state in SF_TS_State]
])
nodes_med_df = pd.DataFrame([
    Node(step_idx=step_idx, state=state).model_dump()
    for step_idx in range(Np) for state in [state for state in MedState]
])

nodes_df = pd.DataFrame(result)
nodes_df_separated = pd.concat([nodes_sfts_df, nodes_med_df], axis=0)

display(nodes_df.head())
display(nodes_df_separated.head())

# Build transition/connection/edges dataframe

# step_idx = 0
# 
# edges_list = []
# for step_idx in range(Np):
#     edges_list = generate_edges(edges_list, step_idx, system='SolarMED', Np=Np)
#     
# # Convert to dataframe
# edges_df = generate_edges_dataframe(edges_list)
# 
# display(edges_df.head())


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SolarMED_State.sf_IDLE_ts_IDLE_med_OFF,step000_000,000,sf_IDLE_ts_IDLE_med_OFF,0,0
1,0,SolarMED_State.sf_IDLE_ts_IDLE_med_GENERATING_...,step000_001,001,sf_IDLE_ts_IDLE_med_GENERATING_VACUUM,0,1
2,0,SolarMED_State.sf_IDLE_ts_IDLE_med_IDLE,step000_002,002,sf_IDLE_ts_IDLE_med_IDLE,0,2
3,0,SolarMED_State.sf_IDLE_ts_IDLE_med_STARTING_UP,step000_003,003,sf_IDLE_ts_IDLE_med_STARTING_UP,0,3
4,0,SolarMED_State.sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,step000_004,004,sf_IDLE_ts_IDLE_med_SHUTTING_DOWN,0,4


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SF_TS_State.IDLE,step000_00,00,IDLE,0,0.0
1,0,SF_TS_State.RECIRCULATING_TS,step000_01,01,RECIRCULATING_TS,0,1.0
2,0,SF_TS_State.HEATING_UP_SF,step000_10,10,HEATING_UP_SF,0,2.0
3,0,SF_TS_State.SF_HEATING_TS,step000_11,11,SF_HEATING_TS,0,3.0
4,1,SF_TS_State.IDLE,step001_00,00,IDLE,1,0.0


In [5]:

# generate_edges_coordinates(nodes_df, edges_df=None)

fig = plot_state_graph(nodes_df, edges_df=None, system='SolarMED', Np=Np, height=1200)

options_avg = round(len(all_paths) / len(SF_TS_State))

@interact(initial_state=[state for state in SolarMED_State], option_idx=(0, options_avg, 1))
def add_path_highlight(initial_state=SolarMED_State.sf_IDLE_ts_IDLE_med_OFF, option_idx=0):

    if initial_state is None:
        # Use random module to choose an integer from 0 to len(all_paths)
        path_idx = np.random.randint(0, len(all_paths))

    else:
        # Find the path that starts from the initial state
        path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
        path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
        print(f"Selected path {path_idx}: {all_paths_str[path_idx]}")

    # Somehow build the path coordinates from the list of all_paths
    path = all_paths[path_idx]

    x = []
    y = []
    for step_idx in range(0, len(path) - 1, 1):
        # If the naming scheme changes for whatever reason it will break
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_df)

        x += x_aux
        y += y_aux

    with fig.batch_update():
        # First deactivate departing highlights
        fig.data[-2].x = None
        fig.data[-2].y = None

        # Then use arriving trace container to include the path
        fig.data[-1].x = x
        fig.data[-1].y = y

# Add a button that will highlight the paths that arrive/leave to/from a random node
# button = widgets.Button(description='Highlight random path')
# 
# button.on_click(add_path_highlight)
# run
fig


interactive(children=(Dropdown(description='initial_state', options=(<SolarMED_State.sf_IDLE_ts_IDLE_med_OFF: …

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#6959CD', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['sf_IDLE_ts_IDLE_med_OFF', 'sf_IDLE_ts_IDLE_med_GENERATING_VACUUM',
                             'sf_IDLE_ts_IDLE_med_IDLE', 'sf_IDLE_ts_IDLE_med_STARTING_UP',
                             'sf_IDLE_ts_IDLE_med_SHUTTING_DOWN', 'sf_IDLE_ts_IDLE_med_ACTIVE',
                             'sf_IDLE_ts_ACTIVE_med_OFF', 'sf_IDLE_ts_ACTIVE_med_GENERATING_VACUUM',
                             'sf_IDLE_ts_ACTIVE_med_IDLE', 'sf_IDLE_ts_ACTIVE_med_STARTING_UP',
                             'sf_IDLE_ts_ACTIVE_med_SHUTTING_DOWN', 'sf_IDLE_ts_ACTIVE_med_ACTIVE',
                             'sf_ACTIVE_ts_IDLE_med_OFF', 'sf_ACTIVE_ts_IDLE_med_GENERATING_VACUUM',
                             'sf_ACTIVE_ts_IDLE_med_IDLE

In [10]:
# Save figure
save_figure(fig, 'SolarMED_state_evaluation_example_v1', 'docs/attachments',
            formats= ['html', 'svg'], width=fig.layout.width, height=fig.layout.height)


2024-05-07 09:10:36.797 | INFO     | phd_visualizations:save_figure:33 - Figure saved in ['docs/attachments']/SolarMED_state_evaluation_example_v1.html
2024-05-07 09:10:37.598 | INFO     | phd_visualizations:save_figure:33 - Figure saved in ['docs/attachments']/SolarMED_state_evaluation_example_v1.svg


In [6]:
fig = plot_state_graph([nodes_sfts_df, nodes_med_df], edges_df=None, system='SolarMED', Np=Np, height=700, width=900)

shift = len([state for state in SF_TS_State])
# options_avg = round(len(all_paths_med) / len([state for state in MedState]))
options_avg = 1000
base_title = "Directed graph of the operating modes evolution in the SolarMED system"


@interact(initial_state=[state for state in SolarMED_State], option_idx=(0, options_avg, 1))
def add_path_highlight(initial_state=SolarMED_State.sf_IDLE_ts_IDLE_med_OFF, option_idx=0):

    if initial_state is None:
        # Use random module to choose an integer from 0 to len(all_paths)
        path_idx = np.random.randint(0, len(all_paths))

    else:
        # Find the path that starts from the initial state
        path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
        path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
        print(f"Selected path {path_idx}: {all_paths_str[path_idx]}")

    # Somehow build the path coordinates from the list of all_paths
    path = all_paths[path_idx]

    x_sf = []; x_med = []
    y_sf = []; y_med = []
    for step_idx in range(0, len(path) - 1, 1):
        # If the naming scheme changes for whatever reason it will break
        
        # SF-TS path
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value[:-1]}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value[:-1]}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_sfts_df)
        x_sf += x_aux; y_sf += y_aux
        
        # MED path
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value[-1]}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value[-1]}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_med_df, y_shift=shift)
        x_med += x_aux; y_med += y_aux

    with fig.batch_update():
        # Use the departing trace container to include the SF-TS path
        fig.data[-2].x = x_sf
        fig.data[-2].y = y_sf

        # Then use arriving trace container to include the MED path
        fig.data[-1].x = x_med
        fig.data[-1].y = y_med
        
        fig.layout.title.text = f'<b>{base_title}</b><br><span style="font-size: 11px;">Selected path {path_idx}: {[state.name for state in all_paths[path_idx]]}</span></br>'

# Add a button that will highlight the paths that arrive/leave to/from a random node
# button = widgets.Button(description='Highlight random path')
# 
# button.on_click(add_path_highlight)
# 
fig

# add_path_highlight(initial_state=SolarMED_State.sf_IDLE_ts_IDLE_med_OFF, option_idx=0)


interactive(children=(Dropdown(description='initial_state', options=(<SolarMED_State.sf_IDLE_ts_IDLE_med_OFF: …

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': 'e68db182-4542-4d9a-bd0d-4979c0c82365',
              'x': array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
              'y': array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])

In [25]:
# Save figure
save_figure(fig, 'SolarMED_state_evaluation_example_v2', 'docs/attachments',
            formats= ['html', 'svg'], width=fig.layout.width, height=fig.layout.height)


2024-05-07 09:38:31.991 | INFO     | phd_visualizations:save_figure:33 - Figure saved in ['docs/attachments']/SolarMED_state_evaluation_example_v2.html
2024-05-07 09:38:32.145 | INFO     | phd_visualizations:save_figure:33 - Figure saved in ['docs/attachments']/SolarMED_state_evaluation_example_v2.svg


In [15]:
fig.layout.title.text


"Directed graph of the operating modes evolution in the SolarMED_State system<br>Selected path 281: ['sf_IDLE_ts_IDLE_med_OFF', 'sf_IDLE_ts_ACTIVE_med_OFF', 'sf_IDLE_ts_IDLE_med_GENERATING_VACUUM', 'sf_IDLE_ts_ACTIVE_med_GENERATING_VACUUM', 'sf_IDLE_ts_IDLE_med_GENERATING_VACUUM']</br>"